In [ ]:
import subprocess
import time
import os
def run_command(command):
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    return result.stdout.strip()

def check_and_kill_session(session_name):
    sessions = run_command("tmux ls")
    if session_name in sessions:
        run_command(f"tmux kill-session -t {session_name}")

def start_new_session(session_name):
    run_command(f"tmux new-session -d -s {session_name}")

def split_into_panes(session_name):
    for i in range(5):
        run_command(f"tmux split-window -h -t {session_name}:0")
        run_command(f"tmux select-layout -t {session_name}:0 tiled")

    for i in range(3):
        run_command(f"tmux split-window -v -t {session_name}:0.{i}")
        run_command(f"tmux select-layout -t {session_name}:0 tiled")

def connect_to_servers_in_panes(session_name, server_ids):
    for i, server_id in enumerate(server_ids):
        pane_id = f"{session_name}:0.{i}"
        password = 'scope'
        ssh_command = f"sshpass -p {password} ssh {server_id}"

        run_command(f"tmux send-keys -t {pane_id} '{ssh_command}' C-m")

def run_commands_in_panes(session_name, commands):
    for pane_id, command in commands.items():
        run_command(f"tmux send-keys -t {session_name}:0.{pane_id} '{command}' C-m")

def run_command_all_panes(session_name, command):
    panes = run_command(f"tmux list-panes -t {session_name} -F '#P'").split('\n')
    print(panes)
    for pane_id in panes:
        run_command(f"tmux send-keys -t {session_name}:0.{pane_id} '{command}' C-m")


def download_files(remote_host, flag, local_base_path, remote_path = '', password='scope'):
    remote_user = 'root'
    if (remote_path == ''):
        remote_path = '/root/node_check'
        remote_file = f"{remote_path}/output.txt"
        os.system(f"mkdir -p {local_base_path}/{flag}")
        local_file = f"{local_base_path}/{flag}/{remote_host}.txt"
        scp_command = f"sshpass -p {password} scp {remote_user}@{remote_host}:{remote_file} {local_file}"
    else:
        os.system(f"mkdir -p {local_base_path}/{flag}")
        local_file = f"{local_base_path}/{flag}/{remote_host}"
        scp_command = f"sshpass -p {password} scp -r {remote_user}@{remote_host}:{remote_path} {local_file}"
    
    try:
        result = subprocess.run(scp_command, shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            print(result.stdout)
        else:
            print(f"Server: {remote_host}, Error: {result.stderr}")
    except Exception as e:
        print(f'Server: {remote_host}, Error: {e}')


def upload_to_servers(server_ids, local_path, remote_path, password='scope'):
    remote_user = 'root'
    if not local_path.endswith('/'):
        local_path += '/'
    
    for i in server_ids:
        rsync_command = f"sshpass -p {password} rsync -avz {local_path} {remote_user}@{i}:{remote_path}"
        exit_code = os.system(rsync_command)
        if exit_code != 0:
            print(f"An error occurred when communicating with {i}. Exit code: {exit_code}")
        else:
            print(f"Successfully uploaded to {i}")

In [ ]:
session_name = "test"
server_ids = [
    # '029',
    # '030',
    # '031',
    # '032',
    # '033',
    # '034',
    '035',
    '036',
    '037',
    '038',
    '039',
    # '040',
    '041',
    '042',
    # '043',
    '044',
    '046',
    '048',
    # '066',
    # '098',
    # '099',
    # '100',
    # '101',
    # '102',
    # '103',
    # '104',
    # '105',
    # '106',
    # '107',
    # '108',
    # '109',
    # '110',
    # '113',
    # '114',
    # '115',
    # '118',
    # '128'
    ]

server_ids = ['uiuc5g-' + x for x in server_ids]
print(server_ids)
print(len(server_ids))

In [ ]:
def make_connection():
    check_and_kill_session(session_name)
    start_new_session(session_name)
    split_into_panes(session_name)
    time.sleep(1)
    connect_to_servers_in_panes(session_name, server_ids)

Establish All Connections

In [ ]:
make_connection()

Test Running Commands in Selected Nodes (Pane ID: Command)

In [ ]:
commands = {1:'ls', 2:'pwd'}
run_commands_in_panes(session_name, commands)

In [ ]:
command = f'clear'
run_command_all_panes(session_name, command)

Start RF Scenario

In [ ]:
command = 'colosseumcli rf start 1009 -c'
commands = {0:command}
run_commands_in_panes(session_name, commands)

<h3>Check for Faulty Nodes</h3>

Create Directory to Collect Logs

In [ ]:
run_command_all_panes(session_name, 'mkdir -p /root/node_check')
run_command_all_panes(session_name, 'clear')

Run the Radio Check

In [ ]:
radio_check_command = '/usr/local/lib/uhd/examples/rx_ascii_art_dft --freq 1.010e9 --rate 1e6 --gain 20 --ref-lvl 0 --dyn-rng 120 --frame-rate 15 > /root/node_check/output.txt'
run_command_all_panes(session_name, radio_check_command)
time.sleep(15)
kill_command = 'C-c'
run_command_all_panes(session_name, kill_command)

In [ ]:
time.sleep(2)
run_command_all_panes(session_name, '/usr/local/lib/uhd/examples/rx_ascii_art_dft --freq 1.010e9 --rate 1e6 --gain 20 --ref-lvl 0 --dyn-rng 120 --frame-rate 15')
time.sleep(15)
run_command_all_panes(session_name, 'C-c')

In [ ]:
kill_command = 'clear'
run_command_all_panes(session_name, kill_command)

In [ ]:
for i in server_ids:
    print(i)
    flag = 'radio_check'
    # download_files(i, flag, '/Users/mttariq2/Desktop/tmux', )
    download_files(i, flag, '/tmp/colos', )
    # rename each file to 

Analyze Files

In [ ]:
faulty_servers = []
for server in server_ids:
    file_path = f"radio_check/{server}.txt"
    file = open(file_path, 'r')
    signal_strength = 0
    for line in file.readlines():
        if '-100' in line:
            # access next line
            line_len = len(line)
            # get each carachter in line and ensure that more than 80% of the characters are '|'
            if line_len == 0:
                print(f"Server {server} is faulty. Not enough signal > -100db!", end = '  ')
                break
            else:
                # count the number of '|' characters in line
                full_bar_count = line.count('|')
                # calculate the percentage of '|' characters in line
                full_bar_percentage = full_bar_count / line_len
                # if the percentage is greater than 80%, then the signal is strong
                if full_bar_percentage > 0.5:
                    signal_strength += 1
                # if the percentage is less than 80%, then the signal is weak
                else:
                    signal_strength -= 1
    print(f"Server {server} signal strength: {signal_strength}")
    if signal_strength <= 0:
        faulty_servers.append(server)

In [ ]:
print(faulty_servers)
faulty_nodes_index = []
for server in faulty_servers:
    # get the index of the server in the server_ids list
    server_index = server_ids.index(server)
    faulty_nodes_index.append(server_index + 1)

print(faulty_nodes_index)
# remove faulty servers from the server_ids list


print(server_ids)

<h3>Remove Faulty Servers</h3>

In [ ]:
for server in faulty_servers:
    server_ids.remove(server)

<h3>Update Radio Interactive Config file with the updated list of Faullty Nodes </h3>

In [ ]:
# File path
file_path = "/Users/mttariq2/Desktop/tmux/code/scope/radio_api/radio_interactive.conf"

# Read the file and store the content
with open(file_path, 'r') as file:
    lines = file.readlines()

# Replace the line containing "faulty-nodes"
for i, line in enumerate(lines):
    if '"faulty-nodes": ' in line:
        lines[i] = f'  "faulty-nodes": "{faulty_nodes_index}"\n'
        break

# Write the updated content back to the file
with open(file_path, 'w') as file:
    file.writelines(lines)

print("File updated successfully.")

In [ ]:
run_command_all_panes(session_name, 'cd /root/radio_api && clear')
run_command_all_panes(session_name, 'python3 scope_start.py --config-file radio_interactive.conf > /root/node_check/output.txt')

In [ ]:

run_command_all_panes(session_name, 'C-d')

<h1> Code Upload </h1>

Push code from the Local Repo through SCP

In [ ]:
# pass command to create a directory called 'scope_updated'
run_command_all_panes(session_name, 'mkdir -p /root/scope_updated')
# pass command to cd into the directory
run_command_all_panes(session_name, 'cd /root/scope_updated')

local_path = 'code/scope/radio_api/'  # Ensure this path is correct
remote_path = '/root/radio_api/'
run_command_all_panes(session_name, f'rm -r {remote_path}')
run_command_all_panes(session_name, f'mkdir -p {remote_path}')
upload_to_servers(server_ids, local_path, remote_path)

local_path = 'code/scope/radio_code/srslte_config/'  # Ensure this path is correct
remote_path = '/root/radio_code/srslte_config/'
run_command_all_panes(session_name, f'rm -r {remote_path}')
run_command_all_panes(session_name, f'mkdir -p {remote_path}')
upload_to_servers(server_ids, local_path, remote_path)


run_command_all_panes(session_name, 'cd /root/radio_api && clear')
run_command_all_panes(session_name, 'python3 scope_start.py --config-file radio_interactive.conf > /root/node_check/output.txt')
run_commands_in_panes(session_name, {0:'python3 scope_start.py --config-file radio_interactive.conf > /root/node_check/output.txt', 4:'python3 scope_start.py --config-file radio_interactive.conf > /root/node_check/output.txt'})


In [ ]:
for i in server_ids:
    if i not in faulty_servers:
        print(i)
        flag = 'test'
        download_files(i, flag,'/Users/mttariq2/Desktop/tmux')

In [ ]:
for i in server_ids:
    print(i)
    flag = 'logs'
    download_files(i, flag, '/Users/mttariq2/Desktop/tmux', '/logs')

In [ ]:
bs_ids = [1]
ue_ids = {2:1, 3:1, 4:1, 5:8}
faulty_servers = [1]

In [ ]:
# remove faulty nodes from the BS and UE lists.
bs_ids = [x for x in bs_ids if x not in faulty_servers]
keys_to_delete = []
for ue, bs in ue_ids.items():
    if bs in faulty_servers:
        print('UE ID ' + str(ue) + ' is associated with a faulty node: ' + str(bs) + ' - Exiting')
        keys_to_delete.append(ue)
    elif ue in faulty_servers:
        print('UE ID ' + str(ue) + ' is faulty. Exiting')
        keys_to_delete.append(ue)

for key in keys_to_delete:
    del ue_ids[key]

In [ ]:
print(bs_ids)
print(ue_ids)
print(faulty_servers)